# functions

In [9]:
path = %pwd
path

'/root/shared/advent_of_code'

In [10]:
# Python 3.x
import re
import numpy as np
import math
import urllib.request
import itertools

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

def Input(day):
    "Open this day's input file."
    filename = path+'/input{}.txt'.format(day)
    return open(filename)

def transpose(matrix): return zip(*matrix)

def first(iterable): return next(iter(iterable))

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

cat = ''.join

Ø   = frozenset() # Empty set
inf = float('inf')
BIG = 10 ** 999

def grep(pattern, lines):
    "Print lines that match pattern."
    for line in lines:
        if re.search(pattern, line):
            print(line)

def groupby(iterable, key=lambda it: it):
    "Return a dic whose keys are key(it) and whose values are all the elements of iterable with that key."
    dic = defaultdict(list)
    for it in iterable:
        dic[key(it)].append(it)
    return dic

def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c

# 2-D points implemented using (x, y) tuples
def X(point): return point[0]
def Y(point): return point[1]

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (X+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def euclidean_distance(p, q=(0, 0)): 
    "Euclidean (hypotenuse) distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def astar_search(start, h_func, moves_func):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(previous, s)
        for s2 in moves_func(s):
            new_cost = path_cost[s] + 1
            if s2 not in path_cost or new_cost < path_cost[s2]:
                heappush(frontier, (new_cost + h_func(s2), s2))
                path_cost[s2] = new_cost
                previous[s2] = s
    return dict(fail=True, front=len(frontier), prev=len(previous))
                
def Path(previous, s): 
    "Return a list of states that lead to state s, according to the previous dict."
    return ([] if (s is None) else Path(previous, previous[s]) + [s])

In [11]:
# Python 3.x Utility Functions

import re
import numpy as np
import math
import random
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque, abc, OrderedDict
from functools   import lru_cache
from itertools   import (permutations, combinations, chain, cycle, product, islice, 
                         takewhile, zip_longest, count as count_from)
from heapq       import heappop, heappush

identity = lambda x: x
letters  = 'abcdefghijklmnopqrstuvwxyz'

cat = ''.join

Ø   = frozenset() # Empty set
inf = float('inf')
BIG = 10 ** 999

################ Functions for Input, Parsing

# def Input(day, year=2017):
#     "Open this day's input file."
#     return open('data/advent{}/input{}.txt'.format(year, day))
    
def array(lines):
    "Parse an iterable of str lines into a 2-D array. If `lines` is a str, do splitlines."
    if isinstance(lines, str): lines = lines.splitlines()
    return mapt(vector, lines)

def vector(line):
    "Parse a str into a tuple of atoms (numbers or str tokens)."
    return mapt(atom, line.split())

def atom(token):
    "Parse a str token into a number, or leave it as a str."
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return token

################ Functions on Iterables

def first(iterable, default=None): return next(iter(iterable), default)

def first_true(iterable, pred=None, default=None):
    """Returns the first true value in the iterable.
    If no true value is found, returns *default*
    If *pred* is not None, returns the first item
    for which pred(item) is true."""
    # first_true([a,b,c], default=x) --> a or b or c or x
    # first_true([a,b], fn, x) --> a if fn(a) else b if fn(b) else x
    return next(filter(pred, iterable), default)

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

def upto(iterable, maxval):
    "From a monotonically increasing iterable, generate all the values <= maxval."
    # Why <= maxval rather than < maxval? In part because that's how Ruby's upto does it.
    return takewhile(lambda x: x <= maxval, iterable)

def groupby(iterable, key=identity):
    "Return a dict of {key(item): [items...]} grouping all items in iterable by keys."
    groups = defaultdict(list)
    for item in iterable:
        groups[key(item)].append(item)
    return groups

def grouper(iterable, n, fillvalue=None):
    """Collect data into fixed-length chunks:
    grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"""
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

def overlapping(iterable, n):
    """Generate all (overlapping) n-element subsequences of iterable.
    overlapping('ABCDEFG', 3) --> ABC BCD CDE DEF EFG"""
    if isinstance(iterable, abc.Sequence):
        yield from (iterable[i:i+n] for i in range(len(iterable) + 1 - n))
    else:
        result = deque(maxlen=n)
        for x in iterable:
            result.append(x)
            if len(result) == n:
                yield tuple(result)
                
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    return overlapping(iterable, 2)

def sequence(iterable, type=tuple):
    "Coerce iterable to sequence: leave it alone if it is already a sequence, else make it of type."
    return iterable if isinstance(iterable, abc.Sequence) else type(iterable)

def join(iterable, sep=''):
    "Join the items in iterable, converting each to a string first."
    return sep.join(map(str, iterable))
                
def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c
            
def quantify(iterable, pred=bool):
    "Count how many times the predicate is true."
    return sum(map(pred, iterable))

def shuffled(iterable):
    "Create a new list out of iterable, and shuffle it."
    new = list(iterable)
    random.shuffle(new)
    return new
    
flatten = chain.from_iterable
            
class Set(frozenset):
    "A frozenset, but with a prettier printer."
    def __repr__(self): return '{' + join(sorted(self), ', ') + '}'
    
def canon(items, typ=None):
    "Canonicalize these order-independent items into a hashable canonical form."
    typ = typ or (cat if isinstance(items, str) else tuple)
    return typ(sorted(items))

def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))
            
################ Math Functions
            
def transpose(matrix): return tuple(zip(*matrix))

def isqrt(n):
    "Integer square root (rounds down)."
    return int(n ** 0.5)

def ints(start, end):
    "The integers from start to end, inclusive: range(start, end+1)"
    return range(start, end + 1)

def floats(start, end, step=1.0):
    "Yields from start to end (inclusive), by increments of step."
    m = (1.0 if step >= 0 else -1.0)
    while start * m <= end * m:
        yield start
        start += step
        
def multiply(numbers):
    "Multiply all the numbers together."
    result = 1
    for n in numbers:
        result *= n
    return result

################ 2-D points implemented using (x, y) tuples

def X(point): x, y = point; return x
def Y(point): x, y = point; return y

origin = (0, 0)
UP, DOWN, LEFT, RIGHT = (0, 1), (0, -1), (-1, 0), (1, 0)

def neighbors4(point): 
    "The four neighboring squares."
    x, y = point
    return (          (x, y-1),
            (x-1, y),           (x+1, y), 
                      (x, y+1))

def neighbors8(point): 
    "The eight neighboring squares."
    x, y = point 
    return ((x-1, y-1), (x, y-1), (x+1, y-1),
            (x-1, y),             (x+1, y),
            (x-1, y+1), (x, y+1), (x+1, y+1))

def cityblock_distance(p, q=origin): 
    "Manhatten distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def distance(p, q=origin): 
    "Hypotenuse distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

################ Debugging 

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def grep(pattern, iterable):
    "Print lines from iterable that match pattern."
    for line in iterable:
        if re.search(pattern, line):
            print(line)

################ A* and Breadth-First Search (tracking states, not actions)

def always(value): return (lambda *args: value)

def Astar(start, moves_func, h_func, cost_func=always(1)):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    Path      = lambda s: ([] if (s is None) else Path(previous[s]) + [s])
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(s)
        for s2 in moves_func(s):
            g = path_cost[s] + cost_func(s, s2)
            if s2 not in path_cost or g < path_cost[s2]:
                heappush(frontier, (g + h_func(s2), s2))
                path_cost[s2] = g
                previous[s2] = s

def bfs(start, moves_func, goals):
    "Breadth-first search"
    goal_func = (goals if callable(goals) else lambda s: s in goals)
    return Astar(start, moves_func, lambda s: (0 if goal_func(s) else 1))

# 1

In [ ]:
inp = Input(1).read()[:-1]

In [ ]:
def parse_ints(inp, jump):
    inp = inp + inp[:jump+1]
    x = 0
    for i in range(len(inp)-jump-1):
        if inp[i]==inp[i+jump]:
            x += int(inp[i])
    return x

In [ ]:
assert parse_ints('1122', 1) == 3
assert parse_ints('1111', 1) == 4
assert parse_ints('1234', 1) == 0
assert parse_ints('91212129', 1) == 9

In [ ]:
parse_ints(inp, 1)

In [ ]:
assert parse_ints('1212',     2) == 6
assert parse_ints('1221',     2) == 0
assert parse_ints('123425',   3) == 4
assert parse_ints('123123',   3) == 12
assert parse_ints('12131415', 4) == 4

In [ ]:
parse_ints(inp, len(inp)//2)

# 2

In [ ]:
def parse_2(inp):
    return [int(x) for x in inp.split()]

def max_dif_min(row):
    "Return max - min."
    return max(parse_lines(row)) - min(parse_lines(row))

In [ ]:
foo = '''5 1 9 5
7 5 3
2 4 6 8'''.split('\n')

assert sum(map(p2, foo)) == 18

In [ ]:
array()

In [ ]:
sum(map(max_dif_min, Input(2).readlines()))

In [ ]:
def find_divisor(inp):
    for i, j in (itertools.combinations(sorted(parse_2(inp), reverse=True), 2)):
        if i%j==0:
            return i//j

In [ ]:
foo = '''5 9 2 8
9 4 7 3
3 8 6 5'''.split('\n')

assert sum(map(find_divisor, foo)) == 9

In [ ]:
sum(map(find_divisor, Input(2).readlines()))

# 3

In [ ]:
inp = 368078

In [ ]:
def spiral():
    "Yield the (x, y) coordinates of successive points in an infinite spiral."
    length = 1
    square = [0, 0]
    yield tuple(square)
    while True:
        yield from leg(square, length, RIGHT)
        yield from leg(square, length, UP)
        length += 1
        yield from leg(square, length, LEFT)
        yield from leg(square, length, DOWN)
        length += 1

def leg(square, length, delta):
    "Complete one leg of given length, mutating `square` and yielding a copy at each step."
    for _ in range(length):
        square[:] = (X(square) + X(delta), Y(square) + Y(delta))
        yield tuple(square) 

In [ ]:
nth(spiral(), inp-1)

In [ ]:
cityblock_distance(_)

In [ ]:
def spiralsums():
    "Yield the values of a spiral where each square has the sum of the 8 neighbors."
    value = defaultdict(int)
    for p in spiral():
        value[p] = sum(value[q] for q in neighbors8(p)) or 1
        yield value[p]

In [ ]:
first(x for x in spiralsums() if x > 368078)

In [ ]:
nth(spiralsums(), inp-1)

In [ ]:
def border_numbers(level):
    if level==0: return [1,1,1,1,1]
    size = level*2 + 1
    start = border_numbers(level-1)[4] + 1
    corners = [start] + [(start-1)+(size-1)*i for i in range(1,5)]
    return corners

In [ ]:
def p3_0(inp):
    i = 0
    grid = {0: [1,1,1,1,1]}
    while inp > grid[i][-1]:
        i+=1
        grid[i] = border_numbers(i)
        if i==100000: break

    return 2*i - min(np.abs([x-inp for x in grid[i]]))

In [ ]:
assert p3_0(1) == 0
assert p3_0(12) == 3
assert p3_0(23) == 2
assert p3_0(1024) == 31

In [ ]:
p3_0(inp)

In [ ]:
i = 0
grid = {0: [1,1,1,1,1]}
while inp > grid[i][-1]:
    i+=1
    grid[i] = border_numbers(i)
    if i==10000: break

2*i - min(np.abs([x-inp for x in grid[i]]))

In [ ]:
2

In [ ]:
bisect.bisect(grid[i], inp)-1

In [ ]:
corners_loc = [
    ( i, -i+1),
    ( i,  i),
    (-i,  i),
    (-i, -i),
    ( i, -i)
]

corners_loc

In [ ]:
grid[i]

In [ ]:
max([x-inp for x in grid[i]])

In [ ]:
#delta = 
inp - grid[i][idx-1]

In [ ]:
371-i

In [ ]:
i-235

In [ ]:
import bisect
idx = bisect.bisect(grid[i], inp)
idx

In [ ]:
corners_loc[idx]

In [ ]:
corners_loc[idx][idx%2]

In [ ]:
abs(corners_loc[idx][idx%2] - delta) + i

In [ ]:
corners_loc[idx]#[idx%2]

In [ ]:
corners_loc[idx]

In [ ]:
inp

In [ ]:
grid[i]

In [ ]:
grid[i][idx-1], grid[i][idx]

In [ ]:
size = i*2 + 1
i, size

In [ ]:
x[0], x[1], inp

In [ ]:
# a b
# c d
{
    x[0]         : ( i, -i+1),
    x[0]-1+(size-1)  : ( i,  i),
    x[0]-1+(size-1)*2: (-i,  i),
    x[0]-1+(size-1)*3: (-i, -i),
    x[0]-1+(size-1)*4: ( i, -i),
}

In [ ]:
x

In [ ]:
border_numbers(i)

In [ ]:
(inp-x[0])/size

In [ ]:
x[0]

In [ ]:
i*2-1

In [ ]:
(2423+1)/4-1

# 4

In [ ]:
def compare_words(line):
    line = line.split()
    return len(set(line))==len(line)

In [ ]:
assert compare_words('aa bb cc dd ee') == True
assert compare_words('aa bb cc dd aa') == False
assert compare_words('aa bb cc dd aaa') == True

In [ ]:
sum(map(compare_words, Input(4).readlines()))

In [ ]:
def compare_word_anagrams(line):
    line = [cat(sorted(x)) for x in line.split()]
    return len(set(line))==len(line)

In [ ]:
assert compare_word_anagrams('abcde fghij') == True
assert compare_word_anagrams('abcde xyz ecdab') == False
assert compare_word_anagrams('a ab abc abd abf abj') == True
assert compare_word_anagrams('iiii oiii ooii oooi oooo') == True
assert compare_word_anagrams('oiii ioii iioi iiio') == False

In [ ]:
sum(map(compare_word_anagrams, Input(4).readlines()))

# 5

In [ ]:
def count_jumps(inp):
    i = 0
    x = 0
    while 0 <= x < len(inp):
        inp[x], x = inp[x]+1, x+inp[x]
        i+=1
        if i>1000000: break
    return i

In [ ]:
assert count_jumps([0, 3,  0,  1,  -3]) == 5

In [ ]:
count_jumps(list(mapt(int, Input(5))))

In [ ]:
def count_jumps_2(inp):
    i = 0
    x = 0
    while 0 <= x < len(inp):
        x0 = x
        x += inp[x]
        if inp[x0] >= 3:
            inp[x0] -= 1
        else:
            inp[x0] += 1
        i+=1
        if i>1000000000: break
    return i#, x, inp

In [ ]:
assert count_jumps_2([0, 3,  0,  1,  -3]) == 10

In [ ]:
count_jumps_2(list(mapt(int, Input(5))))

# 6

In [ ]:
inp = mapt(int, Input(6).readline().split())
inp

In [ ]:
def run(inp):
    l = len(inp)
    mem = list(inp)

    states = [mem.copy()]
    while True:
        val = max(mem)
        idx = mem.index(val)
        mem[idx] = 0
        for i in range(val):
            mem[(i%l + idx + 1)%l] += 1
        if mem in states:
            break
        else:
            states.append(mem.copy())
    return len(states), states

In [ ]:
assert run((0,2,7,0)) == 5

In [ ]:
run(inp)

In [ ]:
def run2(inp):
    l = len(inp)
    mem = list(inp)

    states = [mem.copy()]
    while True:
        val = max(mem)
        idx = mem.index(val)
        mem[idx] = 0
        for i in range(val):
            mem[(i%l + idx + 1)%l] += 1
        if mem in states:
            break
        else:
            states.append(mem.copy())
    return len(states) - states.index(mem)

In [ ]:
assert run2((0,2,7,0)) == 4

In [ ]:
run2(inp)

# 7

In [12]:
inp = Input(7).readlines()
inp[:6]

['nzyiue (57)\n',
 'pdmkag (39)\n',
 'bogbg (13)\n',
 'nubay (45)\n',
 'dukzh (17)\n',
 'kpjxln (44) -> dzzbvkv, gzdxgvj, wsocb, jidxg\n']

In [44]:
def parse(line):
    "Return (name, weight, above)"
    g = re.match(r"(.+) \((\d+)\)(?: -> )?(.+)?", line).groups()
    if g[2] is None:
        return (g[0], int(g[1]), g[2])
    else:
        return (g[0], int(g[1]), g[2].split(', '))

In [45]:
names = [parse(x)[0] for x in inp]

In [46]:
balanced = [parse(x)[2] for x in inp if parse(x)[2]]

In [47]:
balanced_names = [x for sublist in balanced for x in sublist]

In [48]:
[x for x in names if x not in balanced_names]

['hlhomy']

#### part 2

In [49]:
import functools

In [50]:
weights = {}
children = {}
for line in inp:
    weights[parse(line)[0]] = parse(line)[1]
    if parse(line)[2] is None:
        children[parse(line)[0]] = []
    else:
        children[parse(line)[0]] = parse(line)[2]

In [51]:
@functools.lru_cache(None)
def program_wt(name):
    wt = weights[name]
    for n in children[name]:
        wt += program_wt(n)
    return wt

In [52]:
# hlhomy was my root node, assert function weight is sum of all weights
assert program_wt('hlhomy') == sum([int(parse(x)[1]) for x in inp])

In [53]:
# start with smallest weighted tower, since lower ones will also be imbalanced
for name in sorted(names, key=lambda x: program_wt(x)):
    wts = [program_wt(wt) for wt in children[name]]
    if len(set(wts)) > 1:
        odd_man = children[name][wts.index(Counter(wts).most_common()[-1][0])]
        print(int(weights[odd_man]) +
              Counter(wts).most_common()[0][0]
              - Counter(wts).most_common()[-1][0])
        break

1505


# 8

# 9

# 10

# 11

# 12

# 13

# 14

# 15

# 16

# 17

# 18

# 19

# 20

# 21

# 22

# 23

# 24

# 25